# Разработка сентимент-анализа под задачу заказчика

In [2]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV 
from sklearn.grid_search import RandomizedSearchCV 

from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer
import pymorphy2
import re

import sys

reload(sys)
sys.setdefaultencoding('utf8')

# 1. Парсим отзывы о смартфонах.

Парсить будем https://technopoint.ru

In [4]:
%%writefile parser.py
###### Собственно сам парсер. #######

# -*- coding: utf-8 -*-

import requests
import bs4 as bs4
from tqdm import tqdm
import sys

reload(sys)
sys.setdefaultencoding('utf8')


def split_uppercase(string):
    x = ''
    k = 0
    for c in string:
        if k == 0:
            x += c
        elif c.isupper() and not string[k-1].isupper():
            x += ' %s' % c
        else:
            x += c
        k += 1

    return x.strip()


site = 'https://technopoint.ru'
base_url = 'https://technopoint.ru/catalog/17a75b8e16404e77/smartfony'
pages = []

for p in range(1, 33):
    pages.append(base_url + '/?p=' + str(p) + '&i=1&stock=0&order=4')

products_link = []
print "Get Pages:\n"
for page in tqdm(pages):
    req = requests.get(page)
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    links = parser.findAll('a', attrs={'data-role': 'product-cart-link'})
    for link in links:
        products_link.append(site + link['href'] + 'opinion/')


print "\n\nGet Pages With Comments:\n"
pages_with_comments = []
for link in tqdm(products_link):
    req = requests.get(link)
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    ul_pagination = parser.find('ul', {"class": "pagination"})
    comments_pager_last_link = None
    if ul_pagination is not None:
        comments_pager_last_link = ul_pagination.find('li', {"class": "last"}).find('a')

    comments_pages_count = 1
    if comments_pager_last_link is not None:
        comments_pages_count = int(comments_pager_last_link["data-page"]) + 1

    for page_with_comments in range(1, comments_pages_count + 1):
        pages_with_comments.append(link + str(page_with_comments) + '/')

i = 1
print "\n\nProceed Pages With Comments:\n"
for page_with_comments in tqdm(pages_with_comments):
    req = requests.get(page_with_comments)
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    opinion_items = parser.find('div', {"class": "opinion-container"}).findAll('div', {"class": "opinion-item"})
    for opinion_item in opinion_items:
        description = str(opinion_item.find('div', {"class": "descriptions"})).replace("\n", ' ')
        description = split_uppercase(unicode(description))

        grade = opinion_item["data-grade"]

        comment_file = open('./data/' + str(i) + '.txt', 'w')
        comment_file.write(page_with_comments + "\n" + grade + "\n" + description)
        comment_file.close()

        i += 1

print "\n"
print "Total comments: " + str(i)

Парсер только вытащил сырые комментарии с сайта и разложил в файлы. Теперь сформируем csv файл из комментариев.

In [6]:
%%writefile csv_creator.py

# -*- coding: utf-8 -*-

import bs4 as bs4
from tqdm import tqdm
from os import listdir
from os.path import isfile, join
import pandas as pd
import sys

reload(sys)
sys.setdefaultencoding('utf8')

path = './data'
files = [f for f in listdir(path) if isfile(join(path, f))]

texts = []
grades = []
links = []
full_contents = []

for file_name in tqdm(files):
    f = open(path + '/' + file_name)
    content = f.read()
    link, grade, html = content.split("\n")

    parser = bs4.BeautifulSoup(html, 'lxml')
    lis = parser.findAll('li')

    comment_text = ''
    try:
        if int(grade) in [4, 5]:
            comment_text += lis[0].find('span', {'class': 'description-text'}).text
        else:
            comment_text += lis[1].find('span', {'class': 'description-text'}).text
    except AttributeError:
        pass

    try:
        comment_text += ' ' + lis[2].find('span', {'class': 'description-text'}).text
    except (IndexError, AttributeError) as e:
        pass

    texts.append(comment_text)
    links.append(link)
    grades.append(grade)
    full_contents.append(content)

    f.close()


data = pd.DataFrame()
data['grade'] = grades
data['link'] = links
data['text'] = texts
data['content'] = full_contents

data.to_csv('./data.csv', encoding='utf-8')

# 2. Обрабатываем сырые данные.

In [28]:
data = pd.read_csv('./data/data.csv')
del data['Unnamed: 0']
data.head(3)

,grade,link,text,content
0,5,https://technopoint.ru/product/42db0e393aab333...,"большой экран, мощный аккумулятор, режим для ч...",https://technopoint.ru/product/42db0e393aab333...
1,5,https://technopoint.ru/product/da39313444f8333...,Телефоном пользуюсь немного и это скорее перв...,https://technopoint.ru/product/da39313444f8333...
2,4,https://technopoint.ru/product/44b2f37e4035333...,"Цена, брал за 7900 руб. Яркий экран, хорошо ...",https://technopoint.ru/product/44b2f37e4035333...


In [19]:
data.grade.value_counts()

5    8766
4    2408
3    1060
1     652
2     640
Name: grade, dtype: int64

In [24]:
data['label'] = data.grade.apply(lambda x: 0 if x < 4 else 1)
data.label.value_counts()

1    11174
0     2352
Name: label, dtype: int64

In [25]:
corpus = data[['text', 'label']]
drop_indices = np.random.choice(corpus[corpus.label == 1].index, 8750, replace=False)
corpus = corpus.drop(drop_indices)
corpus.label.value_counts()

1    2424
0    2352
Name: label, dtype: int64

In [27]:
# Это функция для обработки строки. На выходе имеем чистую строку из нормализированных слов.

def clean_str(string):
    morph = pymorphy2.MorphAnalyzer()
    
    symbols = [
        ',', '.', '-', '*', '#', ')', '(', '/', '<', '>', ':', '+', '?', '!', '"', '"', '%', '=', '\\', '}'
    ]
    
    for symbol in symbols:
        string = str(string).replace(symbol, ' ')
        
    words = string.split()
    normalized_words = []
    
    for word in words:
        normalized_words.append(morph.parse(unicode(word.strip()))[0].normal_form)
        
    string = unicode(' '.join(normalized_words))
    
    return string

corpus['text'] = corpus.text.apply(clean_str)
corpus.to_csv('./data/data_morph.csv', encoding='utf-8') # Запишем данные так как процедура длительная.

# 3. Строим модель.

In [29]:
corpus = pd.read_csv('./data/data_morph.csv')
del corpus['Unnamed: 0']
corpus.head(3)

,text,label
0,пользоваться телефон 4 месяц не однократно лов...,0
1,отвратительный оболочка от samsung на android ...,0
2,отличный материал изготовление качественный сб...,1


Подберем наилучшие параметры по сетке:

In [31]:
cv = StratifiedKFold(corpus.label, n_folds=10, shuffle=True, random_state=1)

pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer='word')),
    ('classifier', LogisticRegression())
])

pipeline_params = {
    'classifier__penalty': ['l1', 'l2'],
    'vectorizer__stop_words': [stopwords.words('russian'), None],
    'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4)],
    'vectorizer__norm': ['l1', 'l2']
}

grid = GridSearchCV(pipeline, pipeline_params, cv=cv, refit=True, verbose=1, n_jobs=4)

grid.fit(corpus.text.values.astype('U'), corpus.label)
best = grid.best_estimator_
print(
    "Accuracy (TfidfVectorizer + LogisticRegression): {}, params {}" . format(grid.best_score_, grid.best_params_)
)
grid.best_score_

0.9897403685092128

In [33]:
#grid.best_params_

## Обучаем конечную модель

In [34]:
vectorizer = TfidfVectorizer(
    ngram_range=(1, 3), analyzer='word' ,stop_words=stopwords.words('russian'), norm='l2'
)
X_vect = vectorizer.fit_transform(corpus.text.values.astype('U'))

classifier = LogisticRegression(penalty='l2')
fitted = classifier.fit(X_vect, corpus.label)

# 4. Используем модель. Получаем предсказания.

In [35]:
data_test = pd.read_csv('./data/test_data_to_predict.csv')
data_test['text'] = data_test.text.apply(clean_str)

In [36]:
texts_to_predict = data_test.text.values.astype('U')

In [37]:
X_test = vectorizer.transform(texts_to_predict)
predicts = fitted.predict(X_test)
predicts

array([0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 1])

In [38]:
result = pd.DataFrame()

In [39]:
result['Id'] = range(0,100)
result['y'] = predicts

In [40]:
result['y'] = result.y.apply(lambda x: 'neg' if x == 0 else 'pos')

In [41]:
result.head()

,Id,y
0,0,neg
1,1,pos
2,2,neg
3,3,neg
4,4,pos


In [42]:
result.to_csv('./data/result.csv', index=None)